# Restore permissions from backup

This playbook restores group capabilities in CDF from a previously created backup (from **Remove_Capabilities** or **Get_Cognite_Groups**). Authentication uses the same method as the other playbooks.

## Setup path and imports

In [ ]:
import sys
from pathlib import Path

utils_path = (Path("..") / ".." / "utils").resolve()
if str(utils_path) not in sys.path:
    sys.path.insert(0, str(utils_path))

from cognite_auth import client_with_fallback
from group_backup_restore import (
    DEFAULT_ARCHIVE_DIR,
    list_backups,
    load_backup_json,
    restore_groups_from_backup,
)

## Parameters

In [ ]:
# Customer (must match the backup source; must exist in cognite_auth_config.json)
customer = "oxy-oog-dev"

# Archive folder containing groups_backup_*.json / *.xlsx
archive_dir = DEFAULT_ARCHIVE_DIR

# Which backup to use: "latest" or 0-based index from list_backups (newest first)
backup_choice = "latest"

# If True, only print what would be restored; do not call the API
dry_run = True

token_cache_dir = Path.home() / ".cognite" / "token_cache"

## Authenticate

In [ ]:
cache_path = token_cache_dir / f"{customer}.json" if token_cache_dir else None
client = client_with_fallback(customer, cache_path, verbose=True)
print(f"Authenticated for {customer}")

## Choose backup and restore

List backups, load the chosen one, and restore capabilities for the selected customer. Only groups for the current `customer` key in the backup are restored.

In [ ]:
backups = list_backups(archive_dir)
if not backups:
    raise SystemExit(f"No backups found in {archive_dir}")

for i, (excel_p, json_p, ts) in enumerate(backups):
    print(f"  [{i}] {ts}  {json_p.name}")
if backup_choice == "latest":
    chosen_index = 0
else:
    chosen_index = int(backup_choice)
excel_path, json_path, timestamp = backups[chosen_index]
print(f"\nUsing backup: {json_path.name}")

In [ ]:
backup_data = load_backup_json(json_path)
if customer not in backup_data:
    raise SystemExit(f"Customer {customer!r} not found in backup. Keys: {list(backup_data.keys())}")
# Restore only this customer's groups
data_for_customer = {customer: backup_data[customer]}
print(f"Restoring {len(data_for_customer[customer])} groups for {customer} (dry_run={dry_run})")
restore_groups_from_backup(client, data_for_customer, dry_run=dry_run)
if dry_run:
    print("\nSet dry_run = False and re-run to apply restore.")